In [18]:
import os
import requests
import pandas as pd
import unidecode
from tqdm import tqdm

In [19]:
dienthoai = pd.read_csv("Preprocessing_DienThoai.csv")
dienthoai.head(10)

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,dien thoai,41.990.000 ₫,7% Off,11 Đã bán,NaN,Hồ Chí Minh,44.990.000 ₫,20.100 ₫,99%,100%,75%,0,0,0,0,0
1,dien thoai,39.890.000 ₫,10% Off,NaN,NaN,Hà Nội,44.490.000 ₫,38.000 ₫,99%,100%,85%,0,0,0,0,0
2,dien thoai,37.590.001 ₫,16% Off,24 Đã bán,(2),Hồ Chí Minh,44.490.000 ₫,NaN,90%,NaN,100%,0,0,0,0,2
3,dien thoai,36.990.000 ₫,NaN,NaN,NaN,Hồ Chí Minh,NaN,16.500 ₫,98%,82%,100%,0,0,0,0,0
4,dien thoai,32.126.000 ₫,NaN,28 Đã bán,(1),Hồ Chí Minh,38.470.000 ₫,18.300 ₫,98%,100%,100%,0,0,0,0,1
5,dien thoai,31.990.000 ₫,6% Off,1.3K Đã bán,(305),Hồ Chí Minh,33.990.000 ₫,16.500 ₫,91%,99%,100%,2,0,1,0,302
6,dien thoai,31.590.000 ₫,16% Off,21 Đã bán,(2),Hồ Chí Minh,37.490.000 ₫,NaN,90%,NaN,100%,0,0,0,0,3
7,dien thoai,30.090.000 ₫,15% Off,82 Đã bán,(10),Hà Nội,35.990.000 ₫,38.000 ₫,99%,100%,85%,0,0,0,0,10
8,dien thoai,29.517.000 ₫,11% Off,6 Đã bán,NaN,Hồ Chí Minh,32.990.000 ₫,18.300 ₫,99%,100%,100%,0,0,0,0,0
9,dien thoai,29.590.000 ₫,15% Off,11.2K Đã bán,(2631),Hồ Chí Minh,34.999.000 ₫,5.500 ₫,94%,98%,100%,30,3,8,9,2578


In [20]:
# Khởi tạo phiên làm việc với requests
s = requests.Session()

api_base = "https://api.endpoints.anyscale.com/v1"
# Replace with long-lived credentials for production
token = "esecret_y2flezatttwqezaw86fsemvvv2"
url = f"{api_base}/chat/completions"

In [21]:
def call_api(formatted_data):
    body = {
        "model": "meta-llama/Meta-Llama-3-70B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": "Bạn là 1 trợ lí ảo về AI và có nhiệm vụ chính là chuẩn hóa data một cách chính xác nhất"
            },
            {
                "role": "assistant",
                "content": "Nice to meet you! As a virtual assistant specializing in AI, my primary task is to standardize data with utmost accuracy. I'm here to help ensure that data is consistent, reliable, and ready for analysis or modeling.\n\nTo achieve this, I'll employ various techniques, including:\n\n1. **Data Profiling**: I'll analyze the data to understand its distribution, patterns, and quality. This helps identify errors, inconsistencies, and areas that require attention.\n2. **Data Cleansing**: I'll detect and correct errors, handle missing values, and remove duplicates or irrelevant data. This ensures that the data is accurate, complete, and reliable.\n3. **Data Transformation**: I'll convert data types, perform aggregations, and apply business rules to standardize the data format. This makes it easier to analyze and model the data.\n4. **Data Validation**: I'll check the data against a set of rules, constraints, and formats to ensure it meets the required standards.\n5. **Data Normalization**: I'll organize the data into a consistent structure, making it easier to manage, analyze, and scale.\n\nSome of the specific tasks I can perform to standardize data include:\n\n* Handling date and time formats\n* Standardizing categorical variables (e.g., converting"
            },
            {
                "role": "user",
                "content": "Mỗi mục dữ liệu được ngăn cách bởi dấu |. Không bỏ bất kì 1 mục nào, không thêm bất kỳ mục nào, hãy chuẩn hóa thông tin về dạng:\nkhông dấu, số, số, số, số, không dấu, số, số, số, số, số, số, số, số, số, số\nNếu là None thì kết quả ghi là None.\nNgăn cách các giá trị bằng dấu phẩy.Không hiển thị gì thêm, output trả về đúng 16 cột\n\ndien thoai  |  41.990.000 ₫  |  7% Off  |  11 Đã bán  |  NaN  |  Hồ Chí Minh  |  44.990.000 ₫ \n |  20.100 ₫  |  99%  |  100%  |  75%  |  0  |  0  |  0  |  0  |  0"
            },
            {
                "role": "assistant",
                "content": "Here is the standardized data:\n\ndien thoai,41990000,7,11,None,Ho Chi Minh,44990000,20100,99,100,75,0,0,0,0,0"
            },
            {
                "role": "user",
                "content": f"{formatted_data}"
            },
        ],
        "temperature": 0,
        "max_tokens": 256,
        "top_p": 1,
        "frequency_penalty": 0
    }
    response = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body)
    return response.json()

In [22]:
#Chuyển row thành chuỗi ngăn cách nhau bởi dấu |
def format_row(row):
    return '|'.join(map(str, row.values))\

#LLM code
results = []
for index, row in tqdm(dienthoai.iterrows(), total=dienthoai.shape[0], desc="Processing rows"):
    formatted_data = format_row(row)
    api_response = call_api(formatted_data)
    
    if 'choices' in api_response and api_response['choices']:
        cleaned_data = api_response['choices'][0]['message']['content'].strip()
        cleaned_data = cleaned_data.replace('Here is the standardized data:\n\n', '').replace('\n', ',')
        
        print(cleaned_data)
        results.append(cleaned_data.split(','))
    else:
        print(f"No choices found for row {index}")     #Nếu bị lỗi choices thì kết thúc vòng lặp
        break

Processing rows:   0%|          | 1/1200 [00:05<1:46:12,  5.31s/it]

dien thoai,41990000,7,11,None,Ho Chi Minh,44990000,20100,99,100,75,0,0,0,0,0


Processing rows:   0%|          | 2/1200 [00:08<1:21:18,  4.07s/it]

dien thoai,39890000,10,None,None,Ha Noi,44490000,38000,99,100,85,0,0,0,0,0


Processing rows:   0%|          | 3/1200 [00:11<1:15:37,  3.79s/it]

...

dien thoai,9490000,21,None,None,Ho Chi Minh,11990000,18300,85,None,100,0,0,0,0,0


Processing rows:  66%|██████▌   | 786/1200 [41:10<20:55,  3.03s/it]

dien thoai,9480000,14,None,None,Ho Chi Minh,10990000,0,100,None,100,0,0,0,0,0


Processing rows:  66%|██████▌   | 786/1200 [41:12<21:42,  3.15s/it]

No choices found for row 786


In [24]:
cleaned_df = pd.DataFrame(results, columns=dienthoai.columns)
cleaned_df

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,dien thoai,41990000,7,11,None,Ho Chi Minh,44990000,20100,99,100,75,0,0,0,0,0
1,dien thoai,39890000,10,None,None,Ha Noi,44490000,38000,99,100,85,0,0,0,0,0
2,dien thoai,37590001,16,24,2,Ho Chi Minh,44490000,None,90,None,100,0,0,0,0,2
3,dien thoai,36990000,None,None,None,Ho Chi Minh,None,16500,98,82,100,0,0,0,0,0
4,dien thoai,32126000,NaN,28,1,Ho Chi Minh,38470000,18300,98,100,100,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,dien thoai,9500000,None,None,None,Nam Dinh,None,37700,87,80,100,0,0,0,0,0
782,dien thoai,9500000,32,None,None,Ha Noi,13900000,0,75,None,None,0,0,0,0,0
783,dien thoai,9499000,10,None,None,Ho Chi Minh,10499000,0,82,None,100,0,0,0,0,0
784,dien thoai,9490000,21,None,None,Ho Chi Minh,11990000,18300,85,None,100,0,0,0,0,0


In [27]:
#Còn thiếu 414 rows
dienthoai_lack = dienthoai.iloc[786:, :]

In [28]:
#Tạo API mới
s = requests.Session()

api_base = "https://api.endpoints.anyscale.com/v1"
# Replace with long-lived credentials for production
token = "esecret_2wf5w5vwg2473u21qejaduj31w"
url = f"{api_base}/chat/completions"

In [29]:
def call_api_lack(formatted_data):
    body = {
    "model": "meta-llama/Meta-Llama-3-70B-Instruct",
    "messages": [
        {
            "role": "user",
            "content": "Bạn là 1 trợ lí ảo về AI và có khả năng chuẩn hóa data một cách chính xác nhất"
        },
        {
            "role": "assistant",
            "content": "Nice to meet you! As a virtual assistant specializing in AI, I can help with data standardization, which is a crucial step in ensuring data quality and consistency. Data standardization involves transforming data into a consistent format to facilitate efficient data processing, analysis, and decision-making.\n\nTo standardize data accurately, I can assist with the following tasks:\n\n1. **Data Profiling**: I can analyze the data to identify patterns, anomalies, and inconsistencies, providing insights into the data's structure and quality.\n2. **Data Cleansing**: I can help remove duplicates, correct errors, and fill in missing values to ensure data accuracy and completeness.\n3. **Data Transformation**: I can convert data from one format to another, such as converting date formats or categorical variables, to ensure consistency across the dataset.\n4. **Data Normalization**: I can normalize data to reduce dimensionality and improve data quality by scaling/transforming numerical variables to a common range.\n5. **Data Validation**: I can validate data against a set of rules or constraints to ensure it conforms to expected formats and patterns.\n6. **Data Standardization**: I can apply standardized formats to data, such as formatting dates, times, or addresses, to ensure consistency across the dataset.\n\nSome common data standardization tasks"
        },
        {
            "role": "user",
            "content": "Mỗi mục dữ liệu được ngăn cách bởi dấu |. Không bỏ bất kì 1 mục nào, không thêm bất kỳ mục nào, hãy chuẩn hóa thông tin về dạng:\nkhông dấu, số, số, số, số, không dấu, số, số, số, số, số, số, số, số, số, số\nNếu là None thì kết quả ghi là None.\nNgăn cách các giá trị bằng dấu phẩy. Không giải thích output, output trả về đúng 16 cột\n\ndien thoai  |  41.990.000 ₫  |  7% Off  |  11 Đã bán  |  NaN  |  Hồ Chí Minh  |  44.990.000 ₫  |  20.100 ₫  |  99%  |  100%  |  75%  |  0  |  0  |  0  |  0  |  0"
        },
        {
            "role": "assistant",
            "content": "Here is the standardized output:\n\ndien thoai,41990000,7,11,None,Ho Chi Minh,44990000,20100,99,100,75,0,0,0,0,0"
        },
        {
            "role": "user",
            "content": f"{formatted_data}"
        },

    ],
    "temperature": 0,
    "max_tokens": 256,
    "top_p": 1,
    "frequency_penalty": 0
    }
    response = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body)
    return response.json()

In [30]:
results_lack = []

for index, row in tqdm(dienthoai_lack.iterrows(), total=dienthoai_lack.shape[0], desc="Processing rows"):
    formatted_data = format_row(row)
    api_response = call_api(formatted_data)
    
    if 'choices' in api_response and api_response['choices']:
        cleaned_data = api_response['choices'][0]['message']['content'].strip()
        cleaned_data = cleaned_data.replace('Here is the standardized data:\n\n', '').replace('\n', ',')
        print(cleaned_data)
        results_lack.append(cleaned_data.split(','))
    else:
        print(f"No choices found for row {index}")
        break

Processing rows:   0%|          | 1/414 [00:04<30:10,  4.38s/it]

dien thoai,9440640,10,20,1,Ha Noi,10489600,None,92,None,33,0,0,0,0,2


Processing rows:   0%|          | 2/414 [00:08<27:16,  3.97s/it]

dien thoai,9390000,17,None,1,Vinh Phuc,11250000,43900,100,100,97,0,0,0,0,1


Processing rows:   1%|          | 3/414 [00:11<24:04,  3.51s/it]

...

dien thoai,8490000,None,None,None,Ho Chi Minh,8990000,18300,85,None,100,0,0,0,0,0


Processing rows: 100%|█████████▉| 413/414 [19:39<00:02,  2.95s/it]

dien thoai,8490000,None,None,None,Da Nang,9990000,0,97,100,None,0,0,0,0,0


Processing rows: 100%|██████████| 414/414 [19:42<00:00,  2.86s/it]

dien thoai,8480000,None,None,None,Ho Chi Minh,8990000,0,97,100,33,0,0,0,0,0


In [33]:
lack_df = pd.DataFrame(results_lack, columns=dienthoai_lack.columns)
lack_df.head()

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,dien thoai,9440640,10,20,1,Ha Noi,10489600,None,92,None,33,0,0,0,0,2
1,dien thoai,9390000,17,None,1,Vinh Phuc,11250000,43900,100,100,97,0,0,0,0,1
2,dien thoai,9351363,13,38,13,Ha Noi,10790000,13700,95,100,100,0,0,0,0,13
3,dien thoai,9350000,38,35,18,Ha Noi,17000000,32700,91,95,100,0,0,0,0,19
4,dien thoai,9300000,None,None,None,Ho Chi Minh,None,16500,None,66,100,0,0,0,0,0


In [34]:
final_data = pd.concat([cleaned_df, lack_df], ignore_index=True)
final_data 

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,dien thoai,41990000,7,11,None,Ho Chi Minh,44990000,20100,99,100,75,0,0,0,0,0
1,dien thoai,39890000,10,None,None,Ha Noi,44490000,38000,99,100,85,0,0,0,0,0
2,dien thoai,37590001,16,24,2,Ho Chi Minh,44490000,None,90,None,100,0,0,0,0,2
3,dien thoai,36990000,None,None,None,Ho Chi Minh,None,16500,98,82,100,0,0,0,0,0
4,dien thoai,32126000,NaN,28,1,Ho Chi Minh,38470000,18300,98,100,100,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,dien thoai,8490000,None,None,None,Ho Chi Minh,9990000,0,97,100,33,0,0,0,0,0
1196,dien thoai,8490000,None,None,None,Ho Chi Minh,None,None,None,None,None,0,0,0,0,0
1197,dien thoai,8490000,None,None,None,Ho Chi Minh,8990000,18300,85,None,100,0,0,0,0,0
1198,dien thoai,8490000,None,None,None,Da Nang,9990000,0,97,100,None,0,0,0,0,0


In [35]:
final_data.to_csv("LLM_DienThoai.csv", index=False)